In [1]:
import mne
from mne.preprocessing import ICA
from mne.datasets import eegbci
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import matplotlib
import os
"""
  * @author: dengyufeng
  * @Created on 2024/8/13 16:42
 """
# 设置matplotlib后端为TkAgg
matplotlib.use('TkAgg')

# 创建主窗口
root = tk.Tk()
root.title("ICA处理GUI")

raw_list = []
ica = None  # 这个变量将用于存储ICA对象

# 默认ICA参数
n_components = 7
max_iter = 1000

# 创建新的窗口来显示ICA图像
def create_ica_plot_window():
    plot_window = tk.Toplevel(root)
    plot_window.title("ICA图像")

    # 计算所需的行数和列数
    n_cols = 5
    n_rows = (n_components + n_cols - 1) // n_cols

    # 创建一个图形和多个子图
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(15, 3 * n_rows))  # 创建多个子图
    canvas = FigureCanvasTkAgg(fig, master=plot_window)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    return fig, axes, canvas

# 选择文件的函数
def load_eeg_files():
    file_format = format_var.get()
    if file_format == "EDF":
        filetypes = [("EDF文件", "*.edf")]
    elif file_format == "BDF":
        filetypes = [("BDF文件", "*.bdf")]
    elif file_format == "FIF":
        filetypes = [("FIF文件", "*.fif")]
    else:
        filetypes = [("所有支持的格式", "*.edf *.bdf *.fif")]
    
    filepaths = filedialog.askopenfilenames(title="选择EEG文件", filetypes=filetypes)
    if filepaths:
        for filepath in filepaths:
            if filepath.endswith('.edf'):
                raw = mne.io.read_raw_edf(filepath, preload=True)
            elif filepath.endswith('.bdf'):
                raw = mne.io.read_raw_bdf(filepath, preload=True)
            elif filepath.endswith('.fif'):
                raw = mne.io.read_raw_fif(filepath, preload=True)
            else:
                tk.Label(root, text=f"不支持的文件格式: {filepath.split('/')[-1]}").pack()
                continue
            
            # 标准化EEG数据
            eegbci.standardize(raw)
            
            montage_name = montage_var.get()
            montage = mne.channels.make_standard_montage(montage_name)
            raw.set_montage(montage)
            raw_list.append(raw)
            tk.Label(root, text=f"已加载: {filepath.split('/')[-1]} 使用montage: {montage_name}").pack()

# 执行ICA的函数
def run_ica():
    global ica  # 声明使用全局变量
    global n_components, max_iter
    if not raw_list:
        tk.Label(root, text="请先加载EEG文件").pack()
        return
    
    n_components = int(n_components_entry.get())
    max_iter = int(max_iter_entry.get())

    # 创建图形窗口
    fig, axes, canvas = create_ica_plot_window()

    for raw in raw_list:
        ica = ICA(n_components=n_components, random_state=97, max_iter=max_iter, method='fastica')
        ica.fit(raw)
        
        # 清除现有的图像
        for ax in axes.flatten():
            ax.clear()
        
        ica.plot_components(inst=raw, axes=axes.flatten()[:n_components], show=False)  # 将图像绘制到指定的轴上
        canvas.draw()  # 更新Canvas中的图像

# 自动识别并排除肌肉伪影的函数
def exclude_muscle_artifacts():
    global ica  # 声明使用全局变量
    if not raw_list:
        tk.Label(root, text="请先加载EEG文件").pack()
        return
    for raw in raw_list:
        ica = ICA(n_components=n_components, random_state=97, max_iter=max_iter, method='fastica')
        ica.fit(raw)
        muscle_idx_auto, scores = ica.find_bads_muscle(raw)
        ica.exclude = muscle_idx_auto
        ica.apply(raw)
        tk.Label(root, text="已排除肌肉伪影的成分").pack()

# 剔除指定成分的函数
def exclude_components():
    global ica  # 声明使用全局变量
    if ica is None:
        tk.Label(root, text="请先运行ICA").pack()
        return
    
    exclude_indices = exclude_entry.get()
    try:
        exclude_indices = [int(idx) for idx in exclude_indices.split()]
    except ValueError:
        tk.Label(root, text="请输入有效的成分索引，用空格分隔").pack()
        return
    
    ica.exclude = exclude_indices
    for raw in raw_list:
        ica.apply(raw)
    tk.Label(root, text=f"已剔除成分: {exclude_indices}").pack()

# 保存处理后的数据的函数
def save_processed_data():
    if not raw_list:
        tk.Label(root, text="请先加载EEG文件").pack()
        return
    
    save_path = save_path_entry.get()
    custom_name = custom_name_entry.get()
    if not save_path or not custom_name:
        tk.Label(root, text="请输入有效的保存路径和文件名").pack()
        return
    
    # 只处理第一个（也是唯一一个）加载的raw对象
    raw = raw_list[0]
    file_path = f"{save_path}/{custom_name}_ICA_{n_components}_{max_iter}_processed.fif"
    
    if os.path.exists(file_path):
        response = messagebox.askyesno("文件已存在", f"文件 {file_path} 已存在，是否覆盖？")
        if not response:
            return
    
    raw.save(file_path, overwrite=True)
    tk.Label(root, text=f"已保存处理后的数据到: {file_path}").pack()


# 关闭窗口的函数
def close_app():
    root.destroy()

# 创建GUI元素
montage_var = tk.StringVar(value='standard_1005')
montage_options = ['standard_1005', 'standard_1020', 'biosemi64', 'easycap-M1']
format_var = tk.StringVar(value='ALL')
format_options = ['ALL', 'EDF', 'BDF', 'FIF']

tk.Label(root, text="选择EEG数据格式:").pack(pady=5)
format_menu = ttk.Combobox(root, textvariable=format_var, values=format_options)
format_menu.pack(pady=5)

tk.Label(root, text="选择montage系统:").pack(pady=5)
montage_menu = ttk.Combobox(root, textvariable=montage_var, values=montage_options)
montage_menu.pack(pady=5)

tk.Label(root, text="选择ICA成分数:").pack(pady=5)
n_components_entry = tk.Entry(root)
n_components_entry.insert(0, "7")  # 默认值为7
n_components_entry.pack(pady=5)

tk.Label(root, text="选择ICA最大迭代次数:").pack(pady=5)
max_iter_entry = tk.Entry(root)
max_iter_entry.insert(0, "1000")  # 默认值为1000
max_iter_entry.pack(pady=5)

tk.Button(root, text="加载EEG文件", command=load_eeg_files).pack(pady=10)
tk.Button(root, text="排除肌肉伪影", command=exclude_muscle_artifacts).pack(pady=10)
tk.Button(root, text="运行ICA", command=run_ica).pack(pady=10)

tk.Label(root, text="输入要剔除的成分索引（用空格分隔）:").pack(pady=5)
exclude_entry = tk.Entry(root)
exclude_entry.pack(pady=5)
tk.Button(root, text="剔除成分", command=exclude_components).pack(pady=10)

tk.Label(root, text="输入保存处理后数据的地址:").pack(pady=5)
save_path_entry = tk.Entry(root)
save_path_entry.pack(pady=5)

tk.Label(root, text="输入自定义文件名:").pack(pady=5)
custom_name_entry = tk.Entry(root)
custom_name_entry.pack(pady=5)

tk.Button(root, text="保存处理后的数据", command=save_processed_data).pack(pady=10)

tk.Button(root, text="关闭", command=close_app).pack(pady=10)

# 运行GUI主循环
root.mainloop()